# HFT Platform x hftbacktest Walkthrough
本筆記演示如何安裝、準備資料、轉檔、回測策略，並用簡單圖表檢視結果。


In [5]:
# 安裝（在倉庫根目錄的 .venv 內已安裝可跳過）
# !pip install -e .


## 1) 準備事件資料
假設有一份 JSONL 正規化事件（BidAsk/Tick），用內建轉換器生成 hftbacktest npz。

In [6]:
import json

import numpy as np

from hft_platform.backtest.convert import convert_jsonl_to_npz

# 生成一小段示例事件（BidAsk + Tick）
sample_jsonl = 'sample_events.jsonl'
events = [
    {
        "type": "BidAsk",
        "symbol": "2330",
        "exch_ts": 1_000_000,
        "bids": [{"price": 5_000_000, "volume": 10}],
        "asks": [{"price": 5_005_000, "volume": 8}],
    },
    {
        "type": "Tick",
        "symbol": "2330",
        "exch_ts": 1_000_100,
        "price": 5_002_000,
        "volume": 2,
    },
]
with open(sample_jsonl, 'w') as f:
    for e in events:
        f.write(json.dumps(e) + '\n')

npz_path = 'sample_feed.npz'
convert_jsonl_to_npz(sample_jsonl, npz_path, scale=10000)
npz = np.load(npz_path)['data']
npz[:3]  # 檢查前幾筆事件


2025-12-10 22:53:13 [info     ] Converted feed to hftbacktest npz count=3 output=sample_feed.npz


array([(3758096385, 1000000, 1000000, 500. , 10., 0, 0, 0.),
       (3489660929, 1000000, 1000000, 500.5,  8., 0, 0, 0.),
       (3221225474, 1000100, 1000100, 500.2,  2., 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

## 2) 以 Demo 策略跑 hftbacktest 回測
透過橋接器將策略事件/Intent 轉成 hftbacktest 訂單。

In [7]:
from hft_platform.backtest.adapter import StrategyHbtAdapter

adapter = StrategyHbtAdapter(
    data_path=npz_path,
    strategy_module='hft_platform.strategies.demo_strategy',
    strategy_class='DemoStrategy',
    strategy_id='demo',
    symbol='2330',
    tick_size=0.01,
    lot_size=1,
    price_scale=10000,
    timeout=0
)
adapter.run()


2025-12-10 22:53:13 [warning  ] Strategy config not found      path=config/strategies.yaml


TypeError: some keyword arguments unexpected

## 3) 使用 hftbacktest Recorder 檢視績效 (示意)
此示例生成的事件很少，Recorder 主要示範存取方式。

In [ ]:
import hftbacktest as hbt

# 簡單示範：手動建 asset + Recorder 並迭代 feed
asset = hbt.BacktestAsset()
asset.tick_size(0.01)
asset.lot_size(1)
asset.data(npz_path)
eng = hbt.HashMapMarketDepthBacktest([asset])
rec = hbt.Recorder(num_assets=1, record_size=100)
while True:
    r = eng.wait_next_feed(include_order_resp=True, timeout=0)
    if r < 0:
        break
    rec.recorder.record(eng)

records = rec.get(0)  # 取第 0 個資產紀錄
records[:5]


TypeError: some keyword arguments unexpected

## 4) 視覺化示例 (價格/倉位)
使用 matplotlib 將 Recorder 資料畫成簡單線圖。

In [ ]:
import matplotlib.pyplot as plt

ts = records['timestamp']
price = records['price']
position = records['position']

fig, ax = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
ax[0].plot(ts, price, label='mid')
ax[0].set_ylabel('Price')
ax[0].legend()

ax[1].step(ts, position, where='post', label='position')
ax[1].set_ylabel('Position')
ax[1].set_xlabel('Timestamp')
ax[1].legend()
plt.tight_layout()
plt.show()


NameError: name 'records' is not defined

## 5) CLI 方式 (可在 notebook 用 "!" 執行)
```bash
python -m hft_platform backtest convert --input data.jsonl --output feed.npz --scale 10000
python -m hft_platform backtest run --data feed.npz --strategy-module ... --strategy-class ... --symbol 2330
```
可將上述步驟包裝成批次或 CI。

## 6) 使用內建 sample npz
可直接載入 `data/sample_feed.npz` 跑策略或檢視事件。

In [ ]:
import numpy as np

npz_sample = np.load('data/sample_feed.npz')['data']
npz_sample[:5]

## 7) 更多圖表：Spread / Imbalance
從 Recorder 資料繪製 spread 與 imbalance（若 Recorder 有欄位，可從 features 衍生）。

In [ ]:
# 若有 Recorder 或 features，可延伸視覺化 spread/imbalance
# 這裡示範用先前 npz_sample 的價格簡單估算 spread/imbalance
import pandas as pd

rows = []
for ev in npz_sample:
    rows.append({
        'exch_ts': ev['exch_ts'],
        'px': ev['px'],
        'qty': ev['qty'],
        'ev': ev['ev']
    })
df = pd.DataFrame(rows)
# 只做示意：顯示前幾行
df.head()